

**GENAI HANDSON**\

ADVANCED PROMPT ENG

---



NAME:SVOJAS A \
SRN: PES2UG23CS633\
SECTION: J


In [ ]:
import os

os.environ.pop("GROQ_API_KEY", None)
os.environ.pop("GOOGLE_API_KEY", None)

print("Environment cleared.")


Environment cleared.


In [ ]:
!pip install -q --upgrade langchain langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 14.3 MB/s eta 0:00:00


In [ ]:
import os
import getpass
from langchain_groq import ChatGroq

# IMPORTANT: Must start with gsk_
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ API Key: ")

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0
)


Enter your GROQ API Key: ··········


In [ ]:
import os
import getpass
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter GROQ key (must start with gsk_): ")

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0
)

print(llm.invoke("Say hello").content)


Enter GROQ key (must start with gsk_): ··········
Hello. How can I assist you today?


In [ ]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"


In [ ]:
prompt_standard = f"Answer this question: {question}"

print("STANDARD:")
print(llm.invoke(prompt_standard).content)


STANDARD:
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


In [ ]:
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("\nCHAIN OF THOUGHT:")
print(llm.invoke(prompt_cot).content)



CHAIN OF THOUGHT:
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the new tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [ ]:
problem = "How can I get my 5-year-old to eat vegetables?"


In [ ]:
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give one unique creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)


In [ ]:
prompt_judge = ChatPromptTemplate.from_template("""
I have three solutions for '{problem}':

1. {sol1}
2. {sol2}
3. {sol3}

Act as a child psychologist. Choose the most sustainable (not bribery) and explain why.
""")


In [ ]:
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("\nTREE OF THOUGHT RESULT:")
print(tot_chain.invoke(problem))



TREE OF THOUGHT RESULT:
As a child psychologist, I would recommend **Solution 2: "Veggie Gardening"** as the most sustainable approach to encourage a 5-year-old to eat vegetables. Here's why:

1. **Ownership and responsibility**: By involving your child in the process of growing their own vegetables, they develop a sense of ownership and responsibility. This can lead to a deeper appreciation for the food they eat and a willingness to try new vegetables.
2. **Hands-on learning**: Gardening and cooking are hands-on activities that allow your child to learn about the process of growing and preparing food. This experiential learning can help them develop a deeper understanding of the food they eat and its nutritional value.
3. **Development of healthy habits**: By involving your child in the process of growing and preparing their own food, you can help them develop healthy eating habits that will last a lifetime. This approach encourages them to try new foods, experiment with different fl

In [ ]:
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)


In [ ]:
prompt_combine = ChatPromptTemplate.from_template("""
Topic: {topic}

Sci-Fi: {draft_scifi}
Romance: {draft_romance}
Horror: {draft_horror}

Create one paragraph combining technology, passion, and fear.
""")


In [ ]:
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("\nGRAPH OF THOUGHT RESULT:")
print(got_chain.invoke("Time Travel"))



GRAPH OF THOUGHT RESULT:
As Dr. Emma Taylor stood at the controls of her time machine, a fusion of quantum physics and artificial intelligence, she felt the rush of adrenaline coursing through her veins. The sleek, metallic device hummed to life, its holographic display flashing with a countdown to the moment she had been waiting for - a chance to relive the pivotal night that had changed her life forever. But as she set the coordinates and initiated the sequence, a chill ran down her spine. She had been warned about the risks of meddling with the timeline, and the eerie feeling that she was being watched by unseen eyes only added to her growing sense of unease. And yet, she couldn't resist the allure of reuniting with her lost love, Jack, who had been torn from her life by a cruel twist of fate. As the machine whirred to life, Emma felt her heart pounding in her chest, and a whispered warning echoed in her mind: "Be careful what you wish for, for in the world of time travel, the past